In [ ]:
mkdir qiime2-ghana-FR
cd qiime2-ghana-FR
mv rep-seqs.qzv rep-seqs.zip
unzip rep-seqs.zip
unzip -q GHANA-FR.zip
qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path ghana-FR-manifest \
  --output-path paired-end-demux.qza \
  --input-format PairedEndFastqManifestPhred33V2
#output
Imported ghana-FR-manifest as PairedEndFastqManifestPhred33V2 to paired-end-demux.qza
#show quality table
qiime demux summarize \
  --i-data paired-end-demux.qza \
  --o-visualization paired-end-demux.qzv
#output
Saved Visualization to: paired-end-demux.qzv

In [ ]:
#dada2
time qiime dada2 denoise-paired \
  --i-demultiplexed-seqs paired-end-demux.qza \
  --p-trim-left-f 20 \
  --p-trim-left-r 22 \
  --p-trunc-len-f 268 \
  --p-trunc-len-r 261 \
  --o-table table.qza \
  --o-representative-sequences rep-seqs.qza \
  --o-denoising-stats denoising-stats.qza
#output
Saved FeatureTable[Frequency] to: table.qza
Saved FeatureData[Sequence] to: rep-seqs.qza
Saved SampleData[DADA2Stats] to: denoising-stats.qza
    real	119m23.738s
    user	114m43.979s
    sys	2m24.376s
#visulation
qiime feature-table summarize \
  --i-table table.qza \
  --o-visualization table.qzv \
  --m-sample-metadata-file sample-metadata.tsv 

qiime feature-table tabulate-seqs \
  --i-data rep-seqs.qza \
  --o-visualization rep-seqs.qzv

qiime metadata tabulate \
  --m-input-file denoising-stats.qza \
  --o-visualization denoising-stats.qzv
#output
Saved Visualization to: table.qzv
Saved Visualization to: rep-seqs.qzv
Saved Visualization to: denoising-stats.qzv

In [ ]:
#filter rare ASVs mean=66,924.10204081633 *0.1%=67
time qiime feature-table filter-features \
  --i-table table.qza \
  --p-min-frequency 67 \
  --o-filtered-table frequency-filtered-table.qza

qiime feature-table summarize \
  --i-table frequency-filtered-table.qza \
  --o-visualization frequency-filtered-table.qzv \
  --m-sample-metadata-file sample-metadata.tsv 

In [ ]:
# filter seqs according to the filtered-table
time qiime feature-table filter-seqs \
  --i-data rep-seqs.qza \
  --i-table frequency-filtered-table.qza \
  --p-no-exclude-ids \
  --o-filtered-data frequency-filtered-rep-seqs.qza   
qiime feature-table tabulate-seqs \
  --i-data frequency-filtered-rep-seqs.qza \
  --o-visualization frequency-filtered-rep-seqs.qzv    

In [ ]:
#remove non-AMF
#95-90
time qiime feature-classifier classify-consensus-blast \
--i-query frequency-filtered-rep-seqs.qza \
--i-reference-reads SSU_Blast_ref_seqs.qza \
--i-reference-taxonomy ref_seqs_Blast_Taxonomy.qza \
--p-query-cov 0.95 \ #query coverage
--p-perc-identity 0.90 \ #percent identity
--p-maxaccepts 1 \ #only one will be matched. Here if didn't set, could be more than one mach, then it's hard to assign to genus.For example: if at genus level, funneliformis and Glomus are matched, then this sequence will assigned to only family level
--o-classification Blast_rep_taxonomy-CQ-95-90-1.qza
qiime metadata tabulate \
  --m-input-file Blast_rep_taxonomy-CQ-95-90-1.qza \
  --o-visualization Blast_rep_taxonomy-CQ-95-90-1.qzv

In [ ]:
#filter-seqs which non-AMF
time qiime taxa filter-seqs \
--i-sequences frequency-filtered-rep-seqs.qza \
--i-taxonomy Blast_rep_taxonomy-CQ-95-90-1.qza \
--p-exclude Unassigned \
--o-filtered-sequences taxonomy_filtered_seqs.qza
qiime feature-table tabulate-seqs \
  --i-data taxonomy_filtered_seqs.qza \
  --o-visualization taxonomy_filtered_seqs.qzv

In [ ]:
#filter-table which non-AMF
time qiime taxa filter-table \
--i-table frequency-filtered-table.qza \
--i-taxonomy Blast_rep_taxonomy-CQ-95-90-1.qza \
--p-exclude Unassigned \
--o-filtered-table taxonomy-filtered-table.qza
qiime feature-table summarize \
  --i-table taxonomy-filtered-table.qza \
  --o-visualization taxonomy-filtered-table.qzv \
  --m-sample-metadata-file sample-metadata.tsv 